# classification

In [1]:
import numpy as np 
import matplotlib.pyplot as plt 

import cv2 as cv 
from PIL import Image 

import time

from pycoral.adapters import classify, common
from pycoral.utils.edgetpu import make_interpreter
from pycoral.utils.dataset import read_label_file

from reachy_sdk import ReachySDK

from reachy_sdk.trajectory import goto
from reachy_sdk.trajectory.interpolation import InterpolationMode

In [2]:
reachy = ReachySDK('localhost')

In [4]:
path_model = '/home/reachy/dev/reachy-tictactoe/reachy_tictactoe/models/ttt-boxes.tflite'
path_label = '/home/reachy/dev/reachy-tictactoe/reachy_tictactoe/models/ttt-boxes.txt'

In [5]:
interpreter = make_interpreter(path_model)
interpreter.allocate_tensors()

labels = read_label_file(path_label)

size = common.input_size(interpreter)

In [6]:
board_cases = np.array((
    ((81, 166, 260, 340), #Coordinates first board cases (top-left corner) (Xbl, Xbr, Ytr, Ybr)
     (166, 258, 260, 340), #Coordinates second board cases
     (258, 349, 260, 340),),

    ((74, 161, 340, 432),
     (161, 261, 340, 432),
     (261, 360, 340, 432),),

    ((65, 161, 432, 522),
     (161, 266, 432, 522),
     (266, 365, 432, 522),),
))
# left, right, top, bottom
board_rect = np.array((
    74, 384, 216, 483,
))

In [7]:
im = reachy.right_camera.wait_for_new_frame()
height, width = np.shape(im)[:2]


In [41]:
video = cv.VideoWriter('tuto_classification5.avi',cv.VideoWriter_fourcc('M','J','P','G'), 10, (width,height))

reachy.turn_on('head')
reachy.head.look_at(x=1, y=0, z=0, duration=1.5)  
time.sleep(1.5)

reachy.head.look_at(0.5, 0, -0.50, duration=1.25)
time.sleep(1.25)
start = time.time()

fps_count = 0

while time.time() - start < 15:
    
    im = reachy.right_camera.wait_for_new_frame()
    out = im.copy()

    for row in range(3):
        for col in range(3):
            lx, rx, ly, ry = board_cases[row, col]
            pil_img = Image.fromarray(out[ly:ry, lx:rx]).convert('RGB').resize(size, Image.ANTIALIAS)
            pil_img.save(f'/home/reachy/Desktop/Reachy_Nautilus-main/Notebook/image{row}{col}.png')
            common.set_input(interpreter, pil_img)
            interpreter.invoke()
            result = classify.get_classes(interpreter, top_k=3, score_threshold=0.5)
            if result : 
                label = labels.get(result[0].id)
                cv.putText(out, label, (int((lx+rx)/2)-30, int((ly+ry)/2)), cv.FONT_HERSHEY_SIMPLEX, 0.75, (255,0,0), 2)
                cv.rectangle(out, (lx, ly), (rx, ry), (0, 255, 0), 5)
    video.write(out)
            
video.release()

reachy.head.look_at(0.5, 0, 0, duration=1)
time.sleep(0.2)
#reachy.turn_off('head')
